In [12]:
import cv2
import numpy as np
import time

In [3]:
"""
Reads image and returns the image and its gray scale format. 
"""
def read_img(img_path:str):
    img= cv2.imread(img_path)
    return img,cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [4]:
def show_image(window_names:list, images:list):
    for name,img in zip(window_names,images):
        cv2.imshow(name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Section1

### Line Detection

<p> We will cover 3 different techniques wich are :</p>
<ul>
<li>Hough Lines
<li>Hough Lines Transform
<li>Fast Line Detector
</ul>


In [5]:
coral_img,coral_gray_img= read_img('pics/coralReef.png')
show_image(['Coral REEF', 'Gray Coral REEF'],[ coral_img, coral_gray_img])
# show_image(, coral_gray_img)

In [6]:
sharpen_kernel= (1/12)*np.array([
   [ -1,-2,-1]
   ,[-2,24,-2],
   [-1,-2,-1]])
result= coral_gray_img
## First let's do some preprocessing
blurring= [3,3,5,5]
for i in blurring:
    # result= cv2.filter2D(result,-1,sharpen_kernel)
    result= cv2.medianBlur(result ,i)
    result= cv2.filter2D(result,-1,sharpen_kernel)
# result= cv2.medianBlur(result ,5)

edges= cv2.Canny(result,25,255)
show_image(['Gray Coral REEF','Result', 'EDGE DETECTION'],[  coral_gray_img,result,edges])



In [19]:
from math import pi,cos,sin, floor
currentTime= time.time()
vertical_lines= cv2.HoughLines(edges,1,pi/180,65,min_theta=0,max_theta=pi*10/180)
horizontal_lines= cv2.HoughLines(edges,1,pi/180,65,min_theta=pi*80/180,max_theta=pi*90/180)
print(f'Time Elapsed is {(time.time()-currentTime)*1000} ms')
output_image= coral_img.copy()
def drawLines(lines,image):
    for line in lines:
        line=line[0]
        rho = line[0]
        theta = line [1];
        a = cos(theta)
        b = sin(theta);
        x0 = a*rho
        y0 = b*rho;
        x1 = floor(x0 + 1000*(-b))
        y1 =floor(y0 + 1000*(a))
        x2 =floor(x0 - 1000*(-b))
        y2 = floor(y0 - 1000*(a))
        cv2.line( image,(x1,y1),(x2,y2),(0,0,255), 3, cv2.LINE_AA);
drawLines(vertical_lines,output_image)
drawLines(horizontal_lines,output_image)

show_image(['Gray Image','Edges Image', 'Line Detection Image'], [coral_gray_img,edges,output_image])
    

Time Elapsed is 2.941608428955078 ms


In [22]:
currentTime= time.time()
ines= cv2.HoughLinesP(edges,1,pi/180,65, minLineLength= 0, maxLineGap=0)
print(f'Time Elapsed is {(time.time()-currentTime)*1000} ms')
output_img= coral_img.copy()
for line in lines:
    line=line[0]
    cv2.line(output_img, (line[0],line[1]),(line[2],line[3]),(0,0,255), 3, cv2.LINE_AA);
show_image(['Probabilistic Detection'], [output_image])

    

Time Elapsed is 3.317117691040039 ms


In [37]:
# Create default Fast Line Detector class
fld = cv2.ximgproc.createFastLineDetector()
# Get line vectors from the image
lines = fld.detect(edges)
lines_without_blur = fld.detect(coral_gray_img)

# # Draw lines on the image
line_on_image = fld.drawSegments(coral_img, lines)
line_on_image2 = fld.drawSegments(coral_img, lines_without_blur)

show_image(['Line Detector  On Processed', 'Line detector without Porcessing'], [line_on_image, line_on_image2])

# # Plot
# plt.imshow(line_on_image, interpolation='nearest', aspect='auto')
# plt.show()
# return line_on_image